In [ ]:
import utils
import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import MinMaxScaler
import time

In [ ]:
EPOCHS = 1200
BATCH_SIZE = 32
trading_days = 20
model_type = "SIMPLERNN"

In [ ]:
start = time.time()
raw_df = utils.download()
x = utils.process(df=raw_df, trading_days=trading_days)
utils.test(x=x)
x = np.expand_dims(x, 1)
# normalize the dataset
scaler = MinMaxScaler()
x = scaler.fit_transform(x)
x, y = utils.convert2matrix(data_arr=x, look_back=1)
X_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, shuffle=False)
end = time.time()
print("Datenmodellierung: Zeit gebraucht: ", end-start)
print(X_train.shape, y_train.shape, x_test.shape, y_test.shape)

In [ ]:
start = time.time()
model = Sequential()
# Input layer
model.add(SimpleRNN(10))
#Output Layer
model.add(Dense(units = 1))
model.compile(optimizer="adam",loss="mse",metrics=["mse","mae"])

In [ ]:
early_stop = EarlyStopping(monitor="val_loss", patience=100)
history = model.fit(
    X_train,
    y_train,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    validation_data=(x_test, y_test),
    validation_split=0.2,
    verbose=1,
    shuffle=False,
    callbacks=[early_stop],
)
mse = history.history["mse"]
val_mse = history.history["val_mse"]
loss = history.history["loss"]
val_loss = history.history["val_loss"]
mae = history.history["mae"]


In [ ]:
epochs = range(len(mse))
n_epochs = len(mse)
pred = model.predict(x_test)
pred = np.squeeze(pred,1)
y_test = np.squeeze(y_test,1)
prediction = utils.compute_prediction_frame(y_labels=y_test,predicted=pred)

In [ ]:
utils.scatterplotRegression(df=prediction, model_type=model_type, EPOCHS=n_epochs,BATCH_SIZE=BATCH_SIZE)
utils.error_evaluation(epochs,mse,val_mse,loss,val_loss)
end = time.time()
print("Modelling braucht: ", end-start)
utils.print_error(y_train=y_train,y_test=y_test,train_predict=model.predict(X_train),test_predict=pred)

In [ ]:
from sklearn.metrics import mean_absolute_error

print(mean_absolute_error(y_true=y_test,y_pred=pred))

In [ ]:
model.summary()